In [1]:
import numpy as np
import os
import pandas as pd
import requests

from sklearn.model_selection import train_test_split
from tqdm import tqdm

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

from lightautoml.report.monitoring_deco import MonitoringDeco

In [2]:
N_THREADS = 8 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.9 # Test size for metric check
TIMEOUT = 120 # Time in seconds for automl run
TARGET_NAME = 'TARGET' # Target column name

In [3]:
DATASET_DIR = '../data/'
DATASET_NAME = 'sampled_app_train.csv'
DATASET_FULLNAME = os.path.join(DATASET_DIR, DATASET_NAME)
DATASET_URL = 'https://raw.githubusercontent.com/sberbank-ai-lab/LightAutoML/master/examples/data/sampled_app_train.csv'

In [4]:
if not os.path.exists(DATASET_FULLNAME):
    os.makedirs(DATASET_DIR, exist_ok=True)

    dataset = requests.get(DATASET_URL).text
    with open(DATASET_FULLNAME, 'w') as output:
        output.write(dataset)

In [5]:
data = pd.read_csv(DATASET_FULLNAME)

In [6]:
train_data, test_data = train_test_split(data, 
                                         test_size=TEST_SIZE, 
                                         stratify=data[TARGET_NAME], 
                                         random_state=RANDOM_STATE)

Split test data on chunks:

In [7]:
N = 10
test_data['N_EPOCH'] = np.linspace(0.51, N+0.5, test_data.shape[0]).round().astype(int)

n_epochs = test_data['N_EPOCH'].drop_duplicates()
test_data_split = [test_data[test_data['N_EPOCH'] == i] for i in n_epochs]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# MonitoringDeco usage

In [8]:
task = Task('binary', )
automl = TabularAutoML(task = task, 
                       timeout = TIMEOUT,
                       general_params = {'nested_cv': False, 'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                       tuning_params = {'max_tuning_iter': 20, 'max_tuning_time': 30},
                       lgb_params = {'default_params': {'num_threads': N_THREADS}})

In [9]:
MD = MonitoringDeco(output_path='./lama_monitoring_2',
                    report_file_name='lama_monitoring_report.html', \
                    null_threshold=0.2, feature_shift_threshold=0.2, \
                    model_shift_threshold=0.2)

automl_md = MD(automl)

In [10]:
roles = {'target': TARGET_NAME
        }
oof_pred = automl_md.fit_predict(train_data, roles=roles)

In [11]:
for i in tqdm(range(N)):
    automl_md.update(test_data_split[i])

100%|██████████| 10/10 [00:07<00:00,  1.38it/s]
